# Schiphol code assignment environment setup

This notebook is a utility to setup your conda environments for local development.
For this project we will 3 conda environments,

1. schiphol-snakemake: Snakemake and papermill to execute notebooks as scripts in a pipeline
2. schiphol-py: environment with which to execute Python notebooks with datascience tools like pandas, sklearn, tensorflow, etc.
3. schiphol-r: environment with R packages for exploratory analyses and R time-series forecasting

When using [papermill CLI](https://papermill.readthedocs.io/en/latest/usage-cli.html) we can pass a `--kernel` argument that specifies a kernel to use for executing a target notebook. Together with conda we can make our environments available as a `kernel`, but this requires some repetitive setup. 

This notebook creates all 3 environments and makes them available as a kernel. Then finally tests the environment before we continue with the project.

# Setup

In [2]:
from pathlib import Path

In [3]:
# !conda env create -f ../envs/schiphol-snakemake.yml
# !conda env create -f ../envs/schiphol-py.yml
# !conda env create -f ../envs/schiphol-r.yml

In [12]:
env_file_dir = "../envs"
conda_envs = ["schiphol-snakemake", "schiphol-py", "schiphol-r"]

for conda_env in conda_envs:
    env_file = Path(env_file_dir, conda_env + ".yml")
    
    print(conda_env)
    print(f'python -m ipykernel install --user --name {conda_env} --display-name "Python ({conda_env})"')
    
#     !conda deactivate
#     !conda activate $conda_env
#     !python -m ipykernel install --user --name $conda_env --display-name "Python ($conda_env)"
#     !conda deactivate

!jupyter kernelspec list


schiphol-snakemake
python -m ipykernel install --user --name schiphol-snakemake --display-name "Python (schiphol-snakemake)"
schiphol-py
python -m ipykernel install --user --name schiphol-py --display-name "Python (schiphol-py)"
schiphol-r
python -m ipykernel install --user --name schiphol-r --display-name "Python (schiphol-r)"
Available kernels:
  ghosting-cb           C:\Users\lodew\AppData\Roaming\jupyter\kernels\ghosting-cb
  schiphol-py           C:\Users\lodew\AppData\Roaming\jupyter\kernels\schiphol-py
  schiphol-r            C:\Users\lodew\AppData\Roaming\jupyter\kernels\schiphol-r
  schiphol-snakemake    C:\Users\lodew\AppData\Roaming\jupyter\kernels\schiphol-snakemake
  snakemake-py          C:\Users\lodew\AppData\Roaming\jupyter\kernels\snakemake-py
  python3               C:\Users\lodew\anaconda3\share\jupyter\kernels\python3


TO FINISH